In [1]:
import polars as pl
import numpy as np
from scipy.special import psi
from sklearn.datasets import make_classification
from typing import Any
from time import perf_counter

In [2]:
orig_x, orig_y = make_classification(n_samples = 500_000, n_features = 50, n_informative = 25, n_redundant = 25)
df = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y))
target = "target"
features = df.columns
features.remove(target)

In [3]:
df.head() 

target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,column_36,column_37,column_38,column_39,column_40,column_41,column_42,column_43,column_44,column_45,column_46,column_47,column_48,column_49
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,10.173785,-1.11858,2.155507,0.705208,6.955368,-2.769479,1.916058,-0.70979,-1.153821,2.300933,6.982348,2.688324,-3.18071,-2.992516,2.365757,-1.400715,-11.352503,13.143104,1.959231,-3.06955,-3.204233,5.837659,1.898966,-4.553542,-2.586111,1.231594,-0.810111,-4.464101,-1.488461,1.246076,2.656636,-1.438644,-1.713019,-2.916709,0.690035,-6.329929,-2.570733,1.344588,-5.684562,2.441975,3.53397,4.15454,3.807935,-7.83511,-0.1737,0.029458,13.156869,-1.861721,1.201845,2.842081
1,12.517522,2.62133,-4.715827,-2.918456,1.478878,3.147028,4.472198,-0.305051,1.162969,1.964562,10.099554,9.664954,3.252484,3.562439,-0.325829,1.287543,6.643386,-0.593751,-3.085627,5.535441,1.607828,3.434669,2.060355,-0.432489,-1.704382,4.483131,-1.032465,-0.586128,1.005275,-2.744743,2.841304,-6.977464,5.082543,0.745597,1.154642,-13.225422,-1.947237,-6.261176,-7.226839,-1.475961,-3.422894,-0.927674,10.434821,-13.423115,-0.984904,-2.188109,12.963371,-4.743332,1.539239,3.107079
1,-0.498966,-1.398276,-1.507174,3.046764,-0.821387,2.052285,-3.418228,-5.098557,3.863358,2.564739,-6.316938,-13.855033,21.36269,-1.248535,1.495847,4.409152,-13.877812,6.433902,4.711542,1.888253,-0.567898,-2.427339,-4.35135,-0.939048,1.784105,9.99086,0.520372,-1.255366,-0.726969,3.970325,-12.786957,-9.946054,0.19274,-4.839901,3.633465,-0.970085,0.174188,5.665581,0.429374,0.793187,5.14469,2.383533,-18.51138,-4.672322,-0.551774,3.287602,-4.612326,12.16957,0.62844,0.852981
0,-2.883206,-3.544955,-14.273722,-5.700572,10.087774,-2.629106,-3.659399,3.702356,-2.403011,2.572887,-3.544185,10.276686,-1.927202,-5.888428,2.641902,0.024858,-2.438679,-8.565123,9.605598,-11.864494,4.54011,-2.344376,-5.273097,4.230643,-1.16345,10.409231,8.348659,0.705584,0.606112,1.915426,-5.307459,15.684028,-2.805902,2.769533,0.219508,15.556003,-5.997866,-0.057441,2.885738,3.376325,1.664669,5.846563,3.016313,-1.543464,2.336222,-4.04703,0.274011,-8.022961,-0.022842,-2.444585
0,7.253515,3.33296,-19.528059,0.638773,-0.653934,0.518281,-1.538442,-1.678042,2.463945,4.623361,-5.145615,-5.243384,0.61639,0.394251,0.754972,1.540715,0.387187,11.243432,5.114622,8.131682,-3.819916,2.008766,7.637746,-3.567222,-0.571466,5.523208,-2.618538,-4.292979,-3.791118,-0.979461,6.105241,-2.656769,-1.887843,0.335487,-1.460903,-3.211794,-1.238782,-9.253201,-5.382416,-2.76306,0.728382,1.567115,-9.062302,-11.44736,-1.262877,-2.525246,2.986035,-5.638249,5.659785,-0.81967


In [4]:
from scipy.spatial import KDTree

def estimate_mi_optimized(df:pl.DataFrame, cols:list[str], target:str, k=3, random_state:int=42):

    n = len(df)
    rng = np.random.default_rng(random_state)

    target_col = df.get_column(target).to_numpy().ravel()
    unique_targets = np.unique(target_col)
    # parts = {t:df.filter((pl.col(target) == t)) for t in df[target].unique()}
    # To do: If any part size < k, abort. This gets rid of "points with unique labels" issue too.
    all_masks = {}
    for t in unique_targets:
        all_masks[t] = target_col == t
        if len(df.filter(pl.col(target) == t)) <= k:
            raise ValueError(f"The target class {t} must have more than {k} values in the dataset.")        

    estimates = []
    psi_n_and_k = psi(n) + psi(k)
    for col in cols:
        c = df.get_column(col).to_numpy().reshape(-1,1)
        c = c + (1e-10 * np.mean(c) * rng.standard_normal(size=c.shape))
        radius = np.empty(n)
        label_counts = np.empty(n)
        for t in unique_targets:
            mask = all_masks[t]
            c_masked = c[mask]
            kd1 = KDTree(data=c_masked)
            # dd = distances from the points the the k nearest points. +1 because this starts from 0. It is 1 off from sklearn's kdtree.
            dd, _ = kd1.query(c_masked, k = k + 1, workers=-1)
            radius[mask] = np.nextafter(dd[:, -1], 0)
            label_counts[mask] = np.sum(mask)

        kd2 = KDTree(data=c) 
        m_all = kd2.query_ball_point(c, r = radius, return_length=True, workers=-1)
        estimates.append(
            max(0, psi_n_and_k - np.mean(psi(label_counts) + psi(m_all)))
        )

    output = pl.from_records([cols, estimates], schema=["feature", "estimated_mi"])
    return output
        

In [5]:
start = perf_counter()
print(estimate_mi_optimized(df, target=target, cols=features).sort("estimated_mi",  descending=True))
end = perf_counter()
print(f"Spent {end-start:.2f}s.")


shape: (50, 2)
┌───────────┬──────────────┐
│ feature   ┆ estimated_mi │
│ ---       ┆ ---          │
│ str       ┆ f64          │
╞═══════════╪══════════════╡
│ column_4  ┆ 0.112849     │
│ column_37 ┆ 0.072472     │
│ column_43 ┆ 0.069499     │
│ column_6  ┆ 0.062726     │
│ …         ┆ …            │
│ column_22 ┆ 0.001065     │
│ column_44 ┆ 0.00042      │
│ column_14 ┆ 0.000098     │
│ column_32 ┆ 0.000064     │
└───────────┴──────────────┘
Spent 23.70s.


In [6]:
from sklearn.feature_selection import mutual_info_classif

def estimate_mi_sklearn(df:pl.DataFrame, cols:list[str], target:str, k=3, random_state:int=42):
    mi_estimates = mutual_info_classif(df[cols], df[target]
                        , n_neighbors=k, random_state=random_state, discrete_features=False)

    return pl.from_records([cols, mi_estimates], schema=["feature", "estimated_mi"]).sort("estimated_mi", descending=True)

In [7]:
start = perf_counter()
print(estimate_mi_sklearn(df, target=target, cols=features).sort("estimated_mi",  descending=True))
end = perf_counter()
print(f"Spent {end-start:.2f}s.")

shape: (50, 2)
┌───────────┬──────────────┐
│ feature   ┆ estimated_mi │
│ ---       ┆ ---          │
│ str       ┆ f64          │
╞═══════════╪══════════════╡
│ column_4  ┆ 0.11285      │
│ column_37 ┆ 0.07247      │
│ column_43 ┆ 0.069498     │
│ column_6  ┆ 0.062725     │
│ …         ┆ …            │
│ column_22 ┆ 0.001064     │
│ column_44 ┆ 0.00042      │
│ column_14 ┆ 0.000098     │
│ column_32 ┆ 0.000062     │
└───────────┴──────────────┘
Spent 147.96s.


In [8]:
# Slim down before running timeit.

In [9]:
%%timeit 
estimate_mi_optimized(df.limit(100_000), target=target, cols=features).sort("estimated_mi",  descending=True)

4.14 s ± 64.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
estimate_mi_sklearn(df.limit(100_000), target=target, cols=features).sort("estimated_mi",  descending=True)

22.5 s ± 51.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
import sys
sys.path.append('../src')
from eda.eda_selection import mutual_info

In [12]:
mutual_info(df, conti_cols=features, target=target)

100%|██████████| 50/50 [00:21<00:00,  2.31it/s]


feature,estimated_mi
str,f64
"""column_4""",0.112849
"""column_37""",0.072472
"""column_43""",0.069499
"""column_6""",0.062726
"""column_39""",0.060033
"""column_1""",0.05916
"""column_5""",0.056377
"""column_24""",0.053618
"""column_3""",0.045689
